## Humanoid Omni Environment - Full 31-Seed Study

**Environment**: humanoid_omni (biped, 17 DoF, final xy position descriptor)
**Purpose**: Test Competition-GA generalization to complex bipedal locomotion
**Timeline**: Start at 3:15am, finish ~12:15pm

In [1]:
import os
import json
import time
from datetime import datetime
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import functools
import warnings
warnings.filterwarnings('ignore')

import jax
import jax.numpy as jnp

from qdax.core.dns_ga import DominatedNoveltySearchGA
from qdax.core.dns import DominatedNoveltySearch
import qdax.tasks.brax as environments
from qdax.tasks.brax.env_creators import scoring_function_brax_envs as scoring_function
from qdax.core.neuroevolution.buffers.buffer import QDTransition
from qdax.core.neuroevolution.networks.networks import MLP
from qdax.core.emitters.mutation_operators import isoline_variation
from qdax.core.emitters.standard_emitters import MixingEmitter
from qdax.utils.metrics import CSVLogger, default_qd_metrics

# Configure plotting
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

# Create experiment logs directory
os.makedirs("seed_variability_logs_humanoid_omni", exist_ok=True)

print("Setup complete!")
print(f"Current directory: {os.getcwd()}")
print(f"JAX devices: {jax.devices()}")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Failed to import warp: No module named 'warp'
Failed to import mujoco_warp: No module named 'warp'
Setup complete!
Current directory: /Users/briancf/Desktop/source/EvoAlgsAndSwarm/lib-qdax/QDax/examples
JAX devices: [CpuDevice(id=0)]
Start time: 2025-11-16 07:35:20


## Generate Random Seeds (SAME as ant_omni for consistency)

In [2]:
# Use SAME 31 seeds as ant_omni for direct comparison
np.random.seed(2024)
RANDOM_SEEDS = np.random.randint(1, 100000, size=31).tolist()

print("="*80)
print("USING SAME 31 RANDOM SEEDS AS ANT_OMNI")
print("="*80)
print(f"Seeds: {RANDOM_SEEDS[:10]}... (showing first 10)")
print(f"Total: {len(RANDOM_SEEDS)} seeds")
print("="*80)

# Save seeds
with open('seed_variability_logs_humanoid_omni/random_seeds.json', 'w') as f:
    json.dump({'seeds': RANDOM_SEEDS, 'generation_seed': 2024}, f, indent=2)

USING SAME 31 RANDOM SEEDS AS ANT_OMNI
Seeds: [7817, 52731, 51809, 35457, 47644, 95781, 68031, 49336, 7978, 61378]... (showing first 10)
Total: 31 seeds


## Experiment Configuration - Humanoid Omni

In [3]:
FIXED_PARAMS = {
    'batch_size': 256,  # Match ant_omni and walker2d_uni for consistency
    'env_name': 'humanoid_omni',  # Bipedal humanoid, 17 DoF, xy position descriptor
    'episode_length': 25,
    'num_iterations': 3000,
    'policy_hidden_layer_sizes': (64, 64),
    'population_size': 1024,
    'k': 3,
    'line_sigma': 0.05,
    'iso_sigma': 0.01,
}

MAIN_CONFIGS = [
    # Baseline
    {
        'type': 'baseline',
        'name': 'DNS_baseline',
        'g_n': None,
        'num_ga_children': None,
        'num_ga_generations': None,
    },
    # Frequent GA
    {
        'type': 'dns-ga',
        'name': 'DNS-GA_g300_gen2',
        'g_n': 300,
        'num_ga_children': 2,
        'num_ga_generations': 2,
    },
    # Rare but deep GA
    {
        'type': 'dns-ga',
        'name': 'DNS-GA_g1000_gen4',
        'g_n': 1000,
        'num_ga_children': 2,
        'num_ga_generations': 4,
    },
]

print("="*80)
print("HUMANOID OMNI CONFIGURATION")
print("="*80)
print(f"\nEnvironment: {FIXED_PARAMS['env_name']}")
print(f"  Type: Bipedal humanoid")
print(f"  DoF: 17 (most complex)")
print(f"  Descriptor: Final xy position (2D)")
print(f"  Iterations: {FIXED_PARAMS['num_iterations']}")
print(f"  Seeds: {len(RANDOM_SEEDS)}")

print(f"\nConfigurations:")
for config in MAIN_CONFIGS:
    if config['type'] == 'baseline':
        print(f"  • {config['name']}: No GA")
    else:
        ga_calls = FIXED_PARAMS['num_iterations'] // config['g_n']
        print(f"  • {config['name']}: {ga_calls} GA calls")

total_exp = len(MAIN_CONFIGS) * len(RANDOM_SEEDS)
print(f"\nTotal Experiments: {total_exp}")
print(f"Estimated time (2-parallel): ~{(total_exp / 2) * 13.5 / 60:.1f} hours")
print(f"Expected completion: ~12:15pm")
print("="*80)

HUMANOID OMNI CONFIGURATION

Environment: humanoid_omni
  Type: Bipedal humanoid
  DoF: 17 (most complex)
  Descriptor: Final xy position (2D)
  Iterations: 3000
  Seeds: 31

Configurations:
  • DNS_baseline: No GA
  • DNS-GA_g300_gen2: 10 GA calls
  • DNS-GA_g1000_gen4: 3 GA calls

Total Experiments: 93
Estimated time (2-parallel): ~10.5 hours
Expected completion: ~12:15pm


## Helper Functions (same as ant_omni)

In [4]:
def calculate_ga_overhead_evals(g_n, num_iterations, population_size, num_ga_children, num_ga_generations):
    """Calculate total evaluations performed by Competition-GA."""
    if g_n is None or g_n >= num_iterations:
        return 0, 0, 0
    
    num_ga_calls = num_iterations // g_n
    if num_ga_children == 1:
        offspring_per_call = population_size * num_ga_generations
    else:
        offspring_per_call = population_size * num_ga_children * (num_ga_children**num_ga_generations - 1) // (num_ga_children - 1)
    evals_per_ga_call = offspring_per_call
    total_ga_evals = num_ga_calls * evals_per_ga_call
    return total_ga_evals, num_ga_calls, evals_per_ga_call


def setup_environment(env_name, episode_length, policy_hidden_layer_sizes, batch_size, seed):
    """Initialize environment and policy network."""
    env = environments.create(env_name, episode_length=episode_length)
    reset_fn = jax.jit(env.reset)
    key = jax.random.key(seed)
    
    policy_layer_sizes = policy_hidden_layer_sizes + (env.action_size,)
    policy_network = MLP(
        layer_sizes=policy_layer_sizes,
        kernel_init=jax.nn.initializers.lecun_uniform(),
        final_activation=jnp.tanh,
    )
    
    key, subkey = jax.random.split(key)
    keys = jax.random.split(subkey, num=batch_size)
    fake_batch = jnp.zeros(shape=(batch_size, env.observation_size))
    init_variables = jax.vmap(policy_network.init)(keys, fake_batch)
    
    return env, policy_network, reset_fn, init_variables, key


def create_scoring_function(env, policy_network, reset_fn, episode_length, env_name):
    """Create scoring function for fitness evaluation."""
    def play_step_fn(env_state, policy_params, key):
        actions = policy_network.apply(policy_params, env_state.obs)
        state_desc = env_state.info["state_descriptor"]
        next_state = env.step(env_state, actions)
        
        transition = QDTransition(
            obs=env_state.obs,
            next_obs=next_state.obs,
            rewards=next_state.reward,
            dones=next_state.done,
            actions=actions,
            truncations=next_state.info["truncation"],
            state_desc=state_desc,
            next_state_desc=next_state.info["state_descriptor"],
        )
        return next_state, policy_params, key, transition
    
    descriptor_extraction_fn = environments.descriptor_extractor[env_name]
    scoring_fn = functools.partial(
        scoring_function,
        episode_length=episode_length,
        play_reset_fn=reset_fn,
        play_step_fn=play_step_fn,
        descriptor_extractor=descriptor_extraction_fn,
    )
    
    return scoring_fn


def create_mutation_function(iso_sigma):
    """Create mutation function for Competition-GA."""
    def competition_ga_mutation_fn(genotype, key):
        genotype_flat, tree_def = jax.tree_util.tree_flatten(genotype)
        num_leaves = len(genotype_flat)
        keys = jax.random.split(key, num_leaves)
        keys_tree = jax.tree_util.tree_unflatten(tree_def, keys)
        
        def add_noise(x, k):
            return x + jax.random.normal(k, shape=x.shape) * iso_sigma
        
        mutated = jax.tree_util.tree_map(add_noise, genotype, keys_tree)
        return mutated
    
    return competition_ga_mutation_fn

print("Helper functions loaded!")

Helper functions loaded!


## Single Experiment Runner

In [5]:
def run_single_experiment(config, seed, fixed_params):
    """Run a single experiment with given config and seed."""
    exp_name = f"{config['name']}_seed{seed}"
    
    env, policy_network, reset_fn, init_variables, key = setup_environment(
        fixed_params['env_name'],
        fixed_params['episode_length'],
        fixed_params['policy_hidden_layer_sizes'],
        fixed_params['batch_size'],
        seed
    )
    
    scoring_fn = create_scoring_function(env, policy_network, reset_fn, 
                                        fixed_params['episode_length'],
                                        fixed_params['env_name'])
    
    reward_offset = environments.reward_offset[fixed_params['env_name']]
    metrics_function = functools.partial(
        default_qd_metrics,
        qd_offset=reward_offset * fixed_params['episode_length'],
    )
    
    variation_fn = functools.partial(
        isoline_variation,
        iso_sigma=fixed_params['iso_sigma'],
        line_sigma=fixed_params['line_sigma']
    )
    
    mixing_emitter = MixingEmitter(
        mutation_fn=None,
        variation_fn=variation_fn,
        variation_percentage=1.0,
        batch_size=fixed_params['batch_size']
    )
    
    if config['type'] == 'baseline':
        algorithm = DominatedNoveltySearch(
            scoring_function=scoring_fn,
            emitter=mixing_emitter,
            metrics_function=metrics_function,
            population_size=fixed_params['population_size'],
            k=fixed_params['k'],
        )
    else:
        mutation_fn = create_mutation_function(fixed_params['iso_sigma'])
        algorithm = DominatedNoveltySearchGA(
            scoring_function=scoring_fn,
            emitter=mixing_emitter,
            metrics_function=metrics_function,
            population_size=fixed_params['population_size'],
            k=fixed_params['k'],
            g_n=config['g_n'],
            num_ga_children=config['num_ga_children'],
            num_ga_generations=config['num_ga_generations'],
            mutation_fn=mutation_fn,
        )
    
    key, subkey = jax.random.split(key)
    repertoire, emitter_state, init_metrics = algorithm.init(init_variables, subkey)
    
    log_period = 100
    num_loops = fixed_params['num_iterations'] // log_period
    
    # Log initial metrics - maintain consistent ordering
    log_filename = os.path.join("seed_variability_logs_humanoid_omni", f"{exp_name}_logs.csv")
    csv_logger = CSVLogger(log_filename, header=["coverage", "iteration", "max_fitness", "qd_score", "time"])
    
    init_metrics_formatted = jax.tree.map(lambda x: jnp.array([x]) if x.shape == () else x, init_metrics)
    init_metrics_formatted["iteration"] = jnp.array([0], dtype=jnp.int32)
    init_metrics_formatted["time"] = jnp.array([0.0])
    csv_logger.log(jax.tree.map(lambda x: x[-1] if len(x.shape) > 0 else x, init_metrics_formatted))
    
    if config['type'] == 'baseline':
        scan_state = (repertoire, emitter_state, key)
    else:
        scan_state = (repertoire, emitter_state, key, 1)
    
    start_time_total = time.time()
    
    for i in range(num_loops):
        start_time = time.time()
        
        scan_state, current_metrics = jax.lax.scan(
            algorithm.scan_update,
            scan_state,
            (),
            length=log_period,
        )
        
        timelapse = time.time() - start_time
        
        current_metrics["iteration"] = jnp.arange(
            1 + log_period * i, 1 + log_period * (i + 1), dtype=jnp.int32
        )
        current_metrics["time"] = jnp.repeat(timelapse, log_period)
        
        # Only log the last value, don't accumulate full history
        csv_logger.log(jax.tree.map(lambda x: x[-1], current_metrics))
    
    total_time = time.time() - start_time_total
    
    ga_total_evals, ga_num_calls, ga_evals_per_call = calculate_ga_overhead_evals(
        config.get('g_n'), fixed_params['num_iterations'], fixed_params['population_size'],
        config.get('num_ga_children'), config.get('num_ga_generations')
    )
    
    # Save final repertoire for behavior space visualization
    # Extract final repertoire state from scan_state
    if config['type'] == 'baseline':
        final_repertoire = scan_state[0]  # (repertoire, emitter_state, key)
    else:
        final_repertoire = scan_state[0]  # (repertoire, emitter_state, key, generation_counter)
    
    repertoire_file = os.path.join("seed_variability_logs_humanoid_omni", f"{exp_name}_repertoire.npz")
    jnp.savez(repertoire_file,
        descriptors=final_repertoire.descriptors,
        fitnesses=final_repertoire.fitnesses
    )
    
    result = {
        'config_name': config['name'],
        'config_type': config['type'],
        'seed': seed,
        'g_n': config.get('g_n'),
        'num_ga_generations': config.get('num_ga_generations'),
        'final_qd_score': float(current_metrics['qd_score'][-1]),
        'final_max_fitness': float(current_metrics['max_fitness'][-1]),
        'final_coverage': float(current_metrics['coverage'][-1]),
        'total_time': total_time,
        'ga_overhead_evals': ga_total_evals,
        'log_file': log_filename,
        'repertoire_file': repertoire_file,
    }
    
    # Aggressive cleanup to free device memory
    #del repertoire, emitter_state, final_repertoire, scan_state, current_metrics
    #del algorithm, mixing_emitter, scoring_fn, init_variables, env
    
    # Force JAX to clear device memory
    #import gc
    #gc.collect()
    
    # Clear JAX compilation cache periodically
    #jax.clear_caches()
    
    return result

print("Experiment runner ready!")

Experiment runner ready!


## Run Experiments in 5 Groups (Restart Kernel Between Groups)

**Instructions**: 
1. Run GROUP 1 below (19 experiments, ~20 min)
2. **Restart kernel** (Kernel → Restart Kernel)
3. Re-run cells 1-8 to reload setup and functions
4. Run GROUP 2 (19 experiments, ~20 min)
5. Repeat: Restart kernel → Re-run setup → Run next group

**Why**: Prevents memory accumulation across 93 experiments

**Groups**:
- Group 1: Seeds 0-6, all 3 configs (21 experiments)
- Group 2: Seeds 7-13, all 3 configs (21 experiments)
- Group 3: Seeds 14-20, all 3 configs (21 experiments)
- Group 4: Seeds 21-27, all 3 configs (21 experiments)
- Group 5: Seeds 28-30, all 3 configs (9 experiments)

In [6]:
# ============================================================================
# GROUP 1: Seeds 0-6 (indices 0-6) - 21 experiments
# ============================================================================

GROUP_NUM = 1
SEED_START_IDX = 0
SEED_END_IDX = 7  # Exclusive, so seeds 0-6

print("="*80)
print(f"GROUP {GROUP_NUM}: Seeds {SEED_START_IDX} to {SEED_END_IDX-1}")
print("="*80)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
experiment_queue = []
exp_num = 0

for config in MAIN_CONFIGS:
    for seed_idx in range(SEED_START_IDX, SEED_END_IDX):
        seed = RANDOM_SEEDS[seed_idx]
        exp_num += 1
        experiment_queue.append((exp_num, exp_num, config, seed))

print(f"Total experiments in this group: {len(experiment_queue)}")
print(f"Estimated time: ~{len(experiment_queue) * 1:.1f} minutes")
print("="*80)

GROUP 1: Seeds 0 to 6
Total experiments in this group: 21
Estimated time: ~21.0 minutes


In [ ]:
print("\n" + "="*80)
print(f"RUNNING GROUP {GROUP_NUM} EXPERIMENTS")
print("="*80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

start_time_all = time.time()

all_results = []
errors = []
last_update = time.time()

print(f"\nRunning {len(experiment_queue)} experiments sequentially...\n")

for exp_num, (exp_idx, total_exp, config, seed) in enumerate(experiment_queue, 1):
    try:
        result = run_single_experiment(config, seed, FIXED_PARAMS)
        result['exp_num'] = exp_num
        result['group_num'] = GROUP_NUM
        all_results.append(result)
        print(f"  ✓ Completed: {result['config_name']}, seed={result['seed']}, QD={result['final_qd_score']:.1f}")
    except Exception as e:
        errors.append({'config_name': config['name'], 'seed': seed, 'error': str(e)})
        print(f"  ✗ Failed: {config['name']}, seed={seed}, error: {str(e)}")
    
    # Progress update every 10 seconds
    if time.time() - last_update > 10:
        elapsed = time.time() - start_time_all
        pct = exp_num / len(experiment_queue) * 100
        avg_time = elapsed / exp_num
        remaining_time = (len(experiment_queue) - exp_num) * avg_time / 60
        print(f"📊 Progress: {exp_num}/{len(experiment_queue)} ({pct:.1f}%) | Elapsed: {elapsed/60:.1f}m | Remaining: ~{remaining_time:.1f}m")
        last_update = time.time()

total_time = time.time() - start_time_all

print("\n" + "="*80)
print(f"GROUP {GROUP_NUM} COMPLETE!")
print("="*80)
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total time: {total_time / 60:.1f} minutes")
print(f"Successful: {len(all_results)}/{len(experiment_queue)}")
print(f"Failed: {len(errors)}")

if errors:
    print("\nErrors:")
    for error in errors:
        print(f"  • {error['config_name']}, seed={error['seed']}")

results_file = f"seed_variability_logs_humanoid_omni/group{GROUP_NUM}_results_{timestamp}.json"
with open(results_file, 'w') as f:
    json.dump({
        'results': all_results,
        'errors': errors,
        'total_time': total_time,
        'timestamp': timestamp,
        'group_num': GROUP_NUM,
        'environment': 'humanoid_omni',
    }, f, indent=2)

print(f"\nResults saved to: {results_file}")
print("\n" + "="*80)
print("⚠️  NEXT STEP: Restart kernel, re-run setup cells, then run next group")
print("="*80)


RUNNING GROUP 1 EXPERIMENTS
Start time: 2025-11-16 07:35:20

Running 21 experiments sequentially...

  ✓ Completed: DNS_baseline, seed=7817, QD=125642.4
📊 Progress: 1/21 (4.8%) | Elapsed: 10.9m | Remaining: ~218.5m
  ✓ Completed: DNS_baseline, seed=7817, QD=125642.4
📊 Progress: 1/21 (4.8%) | Elapsed: 10.9m | Remaining: ~218.5m
  ✓ Completed: DNS_baseline, seed=52731, QD=125633.7
📊 Progress: 2/21 (9.5%) | Elapsed: 21.8m | Remaining: ~206.9m
  ✓ Completed: DNS_baseline, seed=52731, QD=125633.7
📊 Progress: 2/21 (9.5%) | Elapsed: 21.8m | Remaining: ~206.9m
  ✓ Completed: DNS_baseline, seed=51809, QD=125657.5
📊 Progress: 3/21 (14.3%) | Elapsed: 32.5m | Remaining: ~195.3m
  ✓ Completed: DNS_baseline, seed=51809, QD=125657.5
📊 Progress: 3/21 (14.3%) | Elapsed: 32.5m | Remaining: ~195.3m
  ✓ Completed: DNS_baseline, seed=35457, QD=125637.9
📊 Progress: 4/21 (19.0%) | Elapsed: 43.5m | Remaining: ~184.7m
  ✓ Completed: DNS_baseline, seed=35457, QD=125637.9
📊 Progress: 4/21 (19.0%) | Elapsed: 43.

---
## GROUP 3: Seeds 14-20 (21 experiments)
**After completing Group 2**: Restart kernel → Re-run cells 1-8 → Run cells below

In [ ]:
# ============================================================================
# GROUP 3: Seeds 14-20 (indices 14-20) - 21 experiments
# ============================================================================

GROUP_NUM = 3
SEED_START_IDX = 14
SEED_END_IDX = 21  # Exclusive, so seeds 14-20

print("="*80)
print(f"GROUP {GROUP_NUM}: Seeds {SEED_START_IDX} to {SEED_END_IDX-1}")
print("="*80)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
experiment_queue = []
exp_num = 0

for config in MAIN_CONFIGS:
    for seed_idx in range(SEED_START_IDX, SEED_END_IDX):
        seed = RANDOM_SEEDS[seed_idx]
        exp_num += 1
        experiment_queue.append((exp_num, exp_num, config, seed))

print(f"Total experiments in this group: {len(experiment_queue)}")
print(f"Estimated time: ~{len(experiment_queue) * 1:.1f} minutes")
print("="*80)

In [ ]:
print("\n" + "="*80)
print(f"RUNNING GROUP {GROUP_NUM} EXPERIMENTS")
print("="*80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

start_time_all = time.time()

all_results = []
errors = []
last_update = time.time()

print(f"\nRunning {len(experiment_queue)} experiments sequentially...\n")

for exp_num, (exp_idx, total_exp, config, seed) in enumerate(experiment_queue, 1):
    try:
        result = run_single_experiment(config, seed, FIXED_PARAMS)
        result['exp_num'] = exp_num
        result['group_num'] = GROUP_NUM
        all_results.append(result)
        print(f"  ✓ Completed: {result['config_name']}, seed={result['seed']}, QD={result['final_qd_score']:.1f}")
    except Exception as e:
        errors.append({'config_name': config['name'], 'seed': seed, 'error': str(e)})
        print(f"  ✗ Failed: {config['name']}, seed={seed}, error: {str(e)}")
    
    # Progress update every 10 seconds
    if time.time() - last_update > 10:
        elapsed = time.time() - start_time_all
        pct = exp_num / len(experiment_queue) * 100
        avg_time = elapsed / exp_num
        remaining_time = (len(experiment_queue) - exp_num) * avg_time / 60
        print(f"📊 Progress: {exp_num}/{len(experiment_queue)} ({pct:.1f}%) | Elapsed: {elapsed/60:.1f}m | Remaining: ~{remaining_time:.1f}m")
        last_update = time.time()

total_time = time.time() - start_time_all

print("\n" + "="*80)
print(f"GROUP {GROUP_NUM} COMPLETE!")
print("="*80)
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total time: {total_time / 60:.1f} minutes")
print(f"Successful: {len(all_results)}/{len(experiment_queue)}")
print(f"Failed: {len(errors)}")

if errors:
    print("\nErrors:")
    for error in errors:
        print(f"  • {error['config_name']}, seed={error['seed']}")

results_file = f"seed_variability_logs_humanoid_omni/group{GROUP_NUM}_results_{timestamp}.json"
with open(results_file, 'w') as f:
    json.dump({
        'results': all_results,
        'errors': errors,
        'total_time': total_time,
        'timestamp': timestamp,
        'group_num': GROUP_NUM,
        'environment': 'humanoid_omni',
    }, f, indent=2)

print(f"\nResults saved to: {results_file}")
print("\n" + "="*80)
print("⚠️  NEXT STEP: Restart kernel, re-run setup cells, then run next group")
print("="*80)

---
## GROUP 4: Seeds 21-27 (21 experiments)
**After completing Group 3**: Restart kernel → Re-run cells 1-8 → Run cells below

In [ ]:
# ============================================================================
# GROUP 4: Seeds 21-27 (indices 21-27) - 21 experiments
# ============================================================================

GROUP_NUM = 4
SEED_START_IDX = 21
SEED_END_IDX = 28  # Exclusive, so seeds 21-27

print("="*80)
print(f"GROUP {GROUP_NUM}: Seeds {SEED_START_IDX} to {SEED_END_IDX-1}")
print("="*80)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
experiment_queue = []
exp_num = 0

for config in MAIN_CONFIGS:
    for seed_idx in range(SEED_START_IDX, SEED_END_IDX):
        seed = RANDOM_SEEDS[seed_idx]
        exp_num += 1
        experiment_queue.append((exp_num, exp_num, config, seed))

print(f"Total experiments in this group: {len(experiment_queue)}")
print(f"Estimated time: ~{len(experiment_queue) * 1:.1f} minutes")
print("="*80)

In [ ]:
print("\n" + "="*80)
print(f"RUNNING GROUP {GROUP_NUM} EXPERIMENTS")
print("="*80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

start_time_all = time.time()

all_results = []
errors = []
last_update = time.time()

print(f"\nRunning {len(experiment_queue)} experiments sequentially...\n")

for exp_num, (exp_idx, total_exp, config, seed) in enumerate(experiment_queue, 1):
    try:
        result = run_single_experiment(config, seed, FIXED_PARAMS)
        result['exp_num'] = exp_num
        result['group_num'] = GROUP_NUM
        all_results.append(result)
        print(f"  ✓ Completed: {result['config_name']}, seed={result['seed']}, QD={result['final_qd_score']:.1f}")
    except Exception as e:
        errors.append({'config_name': config['name'], 'seed': seed, 'error': str(e)})
        print(f"  ✗ Failed: {config['name']}, seed={seed}, error: {str(e)}")
    
    # Progress update every 10 seconds
    if time.time() - last_update > 10:
        elapsed = time.time() - start_time_all
        pct = exp_num / len(experiment_queue) * 100
        avg_time = elapsed / exp_num
        remaining_time = (len(experiment_queue) - exp_num) * avg_time / 60
        print(f"📊 Progress: {exp_num}/{len(experiment_queue)} ({pct:.1f}%) | Elapsed: {elapsed/60:.1f}m | Remaining: ~{remaining_time:.1f}m")
        last_update = time.time()

total_time = time.time() - start_time_all

print("\n" + "="*80)
print(f"GROUP {GROUP_NUM} COMPLETE!")
print("="*80)
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total time: {total_time / 60:.1f} minutes")
print(f"Successful: {len(all_results)}/{len(experiment_queue)}")
print(f"Failed: {len(errors)}")

if errors:
    print("\nErrors:")
    for error in errors:
        print(f"  • {error['config_name']}, seed={error['seed']}")

results_file = f"seed_variability_logs_humanoid_omni/group{GROUP_NUM}_results_{timestamp}.json"
with open(results_file, 'w') as f:
    json.dump({
        'results': all_results,
        'errors': errors,
        'total_time': total_time,
        'timestamp': timestamp,
        'group_num': GROUP_NUM,
        'environment': 'humanoid_omni',
    }, f, indent=2)

print(f"\nResults saved to: {results_file}")
print("\n" + "="*80)
print("⚠️  NEXT STEP: Restart kernel, re-run setup cells, then run GROUP 5 (FINAL)")
print("="*80)

---
## GROUP 5: Seeds 28-30 (9 experiments) - FINAL GROUP
**After completing Group 4**: Restart kernel → Re-run cells 1-8 → Run cells below

In [ ]:
# ============================================================================
# GROUP 5: Seeds 28-30 (indices 28-30) - 9 experiments (FINAL)
# ============================================================================

GROUP_NUM = 5
SEED_START_IDX = 28
SEED_END_IDX = 31  # Exclusive, so seeds 28-30

print("="*80)
print(f"GROUP {GROUP_NUM} (FINAL): Seeds {SEED_START_IDX} to {SEED_END_IDX-1}")
print("="*80)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
experiment_queue = []
exp_num = 0

for config in MAIN_CONFIGS:
    for seed_idx in range(SEED_START_IDX, SEED_END_IDX):
        seed = RANDOM_SEEDS[seed_idx]
        exp_num += 1
        experiment_queue.append((exp_num, exp_num, config, seed))

print(f"Total experiments in this group: {len(experiment_queue)}")
print(f"Estimated time: ~{len(experiment_queue) * 1:.1f} minutes")
print("="*80)

In [ ]:
print("\n" + "="*80)
print(f"RUNNING GROUP {GROUP_NUM} EXPERIMENTS (FINAL)")
print("="*80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

start_time_all = time.time()

all_results = []
errors = []
last_update = time.time()

print(f"\nRunning {len(experiment_queue)} experiments sequentially...\n")

for exp_num, (exp_idx, total_exp, config, seed) in enumerate(experiment_queue, 1):
    try:
        result = run_single_experiment(config, seed, FIXED_PARAMS)
        result['exp_num'] = exp_num
        result['group_num'] = GROUP_NUM
        all_results.append(result)
        print(f"  ✓ Completed: {result['config_name']}, seed={result['seed']}, QD={result['final_qd_score']:.1f}")
    except Exception as e:
        errors.append({'config_name': config['name'], 'seed': seed, 'error': str(e)})
        print(f"  ✗ Failed: {config['name']}, seed={seed}, error: {str(e)}")
    
    # Progress update every 10 seconds
    if time.time() - last_update > 10:
        elapsed = time.time() - start_time_all
        pct = exp_num / len(experiment_queue) * 100
        avg_time = elapsed / exp_num
        remaining_time = (len(experiment_queue) - exp_num) * avg_time / 60
        print(f"📊 Progress: {exp_num}/{len(experiment_queue)} ({pct:.1f}%) | Elapsed: {elapsed/60:.1f}m | Remaining: ~{remaining_time:.1f}m")
        last_update = time.time()

total_time = time.time() - start_time_all

print("\n" + "="*80)
print(f"GROUP {GROUP_NUM} COMPLETE! 🎉")
print("="*80)
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total time: {total_time / 60:.1f} minutes")
print(f"Successful: {len(all_results)}/{len(experiment_queue)}")
print(f"Failed: {len(errors)}")

if errors:
    print("\nErrors:")
    for error in errors:
        print(f"  • {error['config_name']}, seed={error['seed']}")

results_file = f"seed_variability_logs_humanoid_omni/group{GROUP_NUM}_results_{timestamp}.json"
with open(results_file, 'w') as f:
    json.dump({
        'results': all_results,
        'errors': errors,
        'total_time': total_time,
        'timestamp': timestamp,
        'group_num': GROUP_NUM,
        'environment': 'humanoid_omni',
    }, f, indent=2)

print(f"\nResults saved to: {results_file}")
print("\n" + "="*80)
print("✅ ALL 5 GROUPS COMPLETE! All humanoid_omni experiments finished.")
print("📁 Results in: seed_variability_logs_humanoid_omni/group1-5_results_*.json")
print("="*80)

## Quick Results Summary

---
## GROUP 2: Seeds 7-13 (21 experiments)
**After completing Group 1**: Restart kernel → Re-run cells 1-8 → Run cells below

In [ ]:
# ============================================================================
# GROUP 2: Seeds 7-13 (indices 7-13) - 21 experiments
# ============================================================================

GROUP_NUM = 2
SEED_START_IDX = 7
SEED_END_IDX = 14  # Exclusive, so seeds 7-13

print("="*80)
print(f"GROUP {GROUP_NUM}: Seeds {SEED_START_IDX} to {SEED_END_IDX-1}")
print("="*80)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
experiment_queue = []
exp_num = 0

for config in MAIN_CONFIGS:
    for seed_idx in range(SEED_START_IDX, SEED_END_IDX):
        seed = RANDOM_SEEDS[seed_idx]
        exp_num += 1
        experiment_queue.append((exp_num, exp_num, config, seed))

print(f"Total experiments in this group: {len(experiment_queue)}")
print(f"Estimated time: ~{len(experiment_queue) * 1:.1f} minutes")
print("="*80)

In [ ]:
print("\n" + "="*80)
print(f"RUNNING GROUP {GROUP_NUM} EXPERIMENTS")
print("="*80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

start_time_all = time.time()

all_results = []
errors = []
last_update = time.time()

print(f"\nRunning {len(experiment_queue)} experiments sequentially...\n")

for exp_num, (exp_idx, total_exp, config, seed) in enumerate(experiment_queue, 1):
    try:
        result = run_single_experiment(config, seed, FIXED_PARAMS)
        result['exp_num'] = exp_num
        result['group_num'] = GROUP_NUM
        all_results.append(result)
        print(f"  ✓ Completed: {result['config_name']}, seed={result['seed']}, QD={result['final_qd_score']:.1f}")
    except Exception as e:
        errors.append({'config_name': config['name'], 'seed': seed, 'error': str(e)})
        print(f"  ✗ Failed: {config['name']}, seed={seed}, error: {str(e)}")
    
    # Progress update every 10 seconds
    if time.time() - last_update > 10:
        elapsed = time.time() - start_time_all
        pct = exp_num / len(experiment_queue) * 100
        avg_time = elapsed / exp_num
        remaining_time = (len(experiment_queue) - exp_num) * avg_time / 60
        print(f"📊 Progress: {exp_num}/{len(experiment_queue)} ({pct:.1f}%) | Elapsed: {elapsed/60:.1f}m | Remaining: ~{remaining_time:.1f}m")
        last_update = time.time()

total_time = time.time() - start_time_all

print("\n" + "="*80)
print(f"GROUP {GROUP_NUM} COMPLETE!")
print("="*80)
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total time: {total_time / 60:.1f} minutes")
print(f"Successful: {len(all_results)}/{len(experiment_queue)}")
print(f"Failed: {len(errors)}")

if errors:
    print("\nErrors:")
    for error in errors:
        print(f"  • {error['config_name']}, seed={error['seed']}")

results_file = f"seed_variability_logs_humanoid_omni/group{GROUP_NUM}_results_{timestamp}.json"
with open(results_file, 'w') as f:
    json.dump({
        'results': all_results,
        'errors': errors,
        'total_time': total_time,
        'timestamp': timestamp,
        'group_num': GROUP_NUM,
        'environment': 'humanoid_omni',
    }, f, indent=2)

print(f"\nResults saved to: {results_file}")
print("\n" + "="*80)
print("⚠️  NEXT STEP: Restart kernel, re-run setup cells, then run next group")
print("="*80)

In [ ]:
if len(all_results) > 0:
    df = pd.DataFrame(all_results)
    print("="*80)
    print("HUMANOID_OMNI RESULTS SUMMARY")
    print("="*80)
    print(f"\nTotal experiments: {len(df)}")
    print("\nFinal QD Scores by Configuration:")
    print(df.groupby('config_name')['final_qd_score'].agg(['mean', 'std', 'min', 'max']).round(2))
    print("\n" + "="*80)
    print("✓ Results ready for integration with ant_omni and walker2d data")
    print("="*80)